In [96]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install -U gensim==3.8.3
!pip install transformers==4.10.2
!pip install sentencepiece==0.1.96
!pip install gtts


     |████████████████████████████████| 13.9 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [97]:
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [98]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

In [99]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")

In [100]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

device  cuda


In [101]:
stopwords = list(STOP_WORDS)
print(stopwords)

['take', 'beyond', 'no', 'call', 'down', 'wherever', 'seems', 'will', 'by', 'meanwhile', 'eleven', 'fifty', 'below', 'twenty', '’m', 'via', 'us', 'along', 'very', 'anything', 'somewhere', 'never', 'rather', 'most', 'already', "'re", 'in', 'ten', 'before', 'beforehand', 'whatever', 'per', '’s', 'four', 'we', '’d', 'using', 'cannot', 'her', 'noone', 'had', 'therein', 'third', 'his', 'sometimes', 'are', 'yours', 'everything', '‘re', 'hereafter', 'how', 'ours', 'them', 'twelve', 'own', 'done', 'even', 'due', 'hereby', 'she', 'many', 'off', 'hereupon', 'keep', 'also', 'first', 'wherein', 'together', 'forty', 'various', 'your', 'am', 'does', 'and', 'yet', 'thereby', 'were', 'our', 'of', 'here', 'these', 'although', 'always', 'hers', 'top', 'hence', 'latter', 'himself', 'it', 'few', 'could', 'should', 'anywhere', 'sixty', 'such', 'whereas', 'side', 'onto', 'latterly', 'herself', 'their', 'too', 'at', 'became', 'two', 'or', 'been', 'only', 'herein', 'move', "'m", 'moreover', 'whence', 'five', 

In [102]:
nlp = spacy.load('en_core_web_sm')

In [193]:

path= '/content/data.txt' #@param {type:"string"}
with open(path) as file:
    text = file.read()
    print(text)

Musk was born to a Canadian mother and South African father, and raised in Pretoria, South Africa. He briefly attended the University of Pretoria before moving to Canada at age 17 to avoid conscription. He was enrolled at Queen's University and transferred to the University of Pennsylvania two years later, where he received a bachelor's degree in economics and physics, then moved to California in 1995 to attend Stanford University but decided instead to pursue a business career, co-founding the web software company Zip2 with his brother Kimbal. The startup was acquired by Compaq for $307 million in 1999. The same year, Musk co-founded online bank X.com, which merged with Confinity in 2000 to form PayPal. The company was bought by eBay in 2002 for $1.5 billion.

In 2002, Musk founded SpaceX, an aerospace manufacturer and space transport services company, of which he is CEO and CTO. In 2004, he joined electric vehicle manufacturer Tesla Motors, Inc. (now Tesla, Inc.) as chairman and prod

In [194]:
print(summarize(text, ratio=1))

Musk was born to a Canadian mother and South African father, and raised in Pretoria, South Africa.
He briefly attended the University of Pretoria before moving to Canada at age 17 to avoid conscription.
He was enrolled at Queen's University and transferred to the University of Pennsylvania two years later, where he received a bachelor's degree in economics and physics, then moved to California in 1995 to attend Stanford University but decided instead to pursue a business career, co-founding the web software company Zip2 with his brother Kimbal.
The startup was acquired by Compaq for $307 million in 1999.
The same year, Musk co-founded online bank X.com, which merged with Confinity in 2000 to form PayPal. The company was bought by eBay in 2002 for $1.5 billion.
In 2002, Musk founded SpaceX, an aerospace manufacturer and space transport services company, of which he is CEO and CTO.
In 2004, he joined electric vehicle manufacturer Tesla Motors, Inc.
(now Tesla, Inc.) as chairman and produ

In [195]:
topics = keywords(text, words=10)

In [196]:
print(topics)

company
founded
transport
transportation
musk
south
publicized
public
manufacturer
create
jury
tesla


In [197]:
doc = nlp(text)

In [198]:
token = [token.text for token in doc]
print(token)

['Musk', 'was', 'born', 'to', 'a', 'Canadian', 'mother', 'and', 'South', 'African', 'father', ',', 'and', 'raised', 'in', 'Pretoria', ',', 'South', 'Africa', '.', 'He', 'briefly', 'attended', 'the', 'University', 'of', 'Pretoria', 'before', 'moving', 'to', 'Canada', 'at', 'age', '17', 'to', 'avoid', 'conscription', '.', 'He', 'was', 'enrolled', 'at', 'Queen', "'s", 'University', 'and', 'transferred', 'to', 'the', 'University', 'of', 'Pennsylvania', 'two', 'years', 'later', ',', 'where', 'he', 'received', 'a', 'bachelor', "'s", 'degree', 'in', 'economics', 'and', 'physics', ',', 'then', 'moved', 'to', 'California', 'in', '1995', 'to', 'attend', 'Stanford', 'University', 'but', 'decided', 'instead', 'to', 'pursue', 'a', 'business', 'career', ',', 'co', '-', 'founding', 'the', 'web', 'software', 'company', 'Zip2', 'with', 'his', 'brother', 'Kimbal', '.', 'The', 'startup', 'was', 'acquired', 'by', 'Compaq', 'for', '$', '307', 'million', 'in', '1999', '.', 'The', 'same', 'year', ',', 'Musk'

In [199]:
punctuation = punctuation + '\n'
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~






In [200]:
word_frequency = {}
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequency.keys():
                word_frequency[word.text] = 1
            else:
                word_frequency[word.text] += 1

In [201]:
print(word_frequency)

{'Musk': 7, 'born': 1, 'Canadian': 1, 'mother': 1, 'South': 2, 'African': 1, 'father': 1, 'raised': 1, 'Pretoria': 2, 'Africa': 1, 'briefly': 1, 'attended': 1, 'University': 4, 'moving': 1, 'Canada': 1, 'age': 1, '17': 1, 'avoid': 1, 'conscription': 1, 'enrolled': 1, 'Queen': 1, 'transferred': 1, 'Pennsylvania': 1, 'years': 1, 'later': 2, 'received': 2, 'bachelor': 1, 'degree': 1, 'economics': 1, 'physics': 1, 'moved': 1, 'California': 2, '1995': 1, 'attend': 1, 'Stanford': 1, 'decided': 1, 'instead': 1, 'pursue': 1, 'business': 1, 'career': 1, 'co': 4, 'founding': 1, 'web': 1, 'software': 1, 'company': 7, 'Zip2': 1, 'brother': 1, 'Kimbal': 1, 'startup': 1, 'acquired': 2, 'Compaq': 1, '307': 1, 'million': 1, '1999': 1, 'year': 2, 'founded': 5, 'online': 1, 'bank': 1, 'X.com': 1, 'merged': 1, 'Confinity': 1, '2000': 1, 'form': 1, 'PayPal': 1, 'bought': 1, 'eBay': 1, '2002': 2, '1.5': 1, 'billion': 1, 'SpaceX': 1, 'aerospace': 1, 'manufacturer': 2, 'space': 1, 'transport': 2, 'services':

In [202]:
max_frequency = max(word_frequency.values())

In [203]:
max_frequency

7

In [204]:
for word in word_frequency.keys():
    word_frequency[word] = word_frequency[word]/max_frequency

In [205]:
print(word_frequency)

{'Musk': 1.0, 'born': 0.14285714285714285, 'Canadian': 0.14285714285714285, 'mother': 0.14285714285714285, 'South': 0.2857142857142857, 'African': 0.14285714285714285, 'father': 0.14285714285714285, 'raised': 0.14285714285714285, 'Pretoria': 0.2857142857142857, 'Africa': 0.14285714285714285, 'briefly': 0.14285714285714285, 'attended': 0.14285714285714285, 'University': 0.5714285714285714, 'moving': 0.14285714285714285, 'Canada': 0.14285714285714285, 'age': 0.14285714285714285, '17': 0.14285714285714285, 'avoid': 0.14285714285714285, 'conscription': 0.14285714285714285, 'enrolled': 0.14285714285714285, 'Queen': 0.14285714285714285, 'transferred': 0.14285714285714285, 'Pennsylvania': 0.14285714285714285, 'years': 0.14285714285714285, 'later': 0.2857142857142857, 'received': 0.2857142857142857, 'bachelor': 0.14285714285714285, 'degree': 0.14285714285714285, 'economics': 0.14285714285714285, 'physics': 0.14285714285714285, 'moved': 0.14285714285714285, 'California': 0.2857142857142857, '19

In [206]:
for key,value in word_frequency.items():
    if key in topics:
        value += 1 
        word_frequency.update({key:value}) 
print(word_frequency.items())

dict_items([('Musk', 1.0), ('born', 0.14285714285714285), ('Canadian', 0.14285714285714285), ('mother', 0.14285714285714285), ('South', 0.2857142857142857), ('African', 0.14285714285714285), ('father', 0.14285714285714285), ('raised', 0.14285714285714285), ('Pretoria', 0.2857142857142857), ('Africa', 0.14285714285714285), ('briefly', 0.14285714285714285), ('attended', 0.14285714285714285), ('University', 0.5714285714285714), ('moving', 0.14285714285714285), ('Canada', 0.14285714285714285), ('age', 0.14285714285714285), ('17', 0.14285714285714285), ('avoid', 0.14285714285714285), ('conscription', 0.14285714285714285), ('enrolled', 0.14285714285714285), ('Queen', 0.14285714285714285), ('transferred', 0.14285714285714285), ('Pennsylvania', 0.14285714285714285), ('years', 0.14285714285714285), ('later', 0.2857142857142857), ('received', 0.2857142857142857), ('bachelor', 0.14285714285714285), ('degree', 0.14285714285714285), ('economics', 0.14285714285714285), ('physics', 0.1428571428571428

In [207]:
sentence_tokens = [sent for sent  in doc.sents]
print(sentence_tokens)

[Musk was born to a Canadian mother and South African father, and raised in Pretoria, South Africa., He briefly attended the University of Pretoria before moving to Canada at age 17 to avoid conscription., He was enrolled at Queen's University and transferred to the University of Pennsylvania two years later, where he received a bachelor's degree in economics and physics, then moved to California in 1995 to attend Stanford University but decided instead to pursue a business career, co-founding the web software company Zip2 with his brother Kimbal., The startup was acquired by Compaq for $307 million in 1999., The same year, Musk co-founded online bank X.com, which merged with Confinity in 2000 to form PayPal., The company was bought by eBay in 2002 for $1.5 billion., 

In 2002, Musk founded SpaceX, an aerospace manufacturer and space transport services company, of which he is CEO and CTO., In 2004, he joined electric vehicle manufacturer Tesla Motors, Inc. (now Tesla, Inc.) as chairman

In [208]:
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequency.keys():
            if sent not in sentence_scores.keys():
                sentence_scores[sent] = word_frequency[word.text.lower()] 
            else:
                sentence_scores[sent] += word_frequency[word.text.lower()]

In [209]:
sentence_scores

{Musk was born to a Canadian mother and South African father, and raised in Pretoria, South Africa.: 0.5714285714285714,
 He briefly attended the University of Pretoria before moving to Canada at age 17 to avoid conscription.: 0.9999999999999998,
 He was enrolled at Queen's University and transferred to the University of Pennsylvania two years later, where he received a bachelor's degree in economics and physics, then moved to California in 1995 to attend Stanford University but decided instead to pursue a business career, co-founding the web software company Zip2 with his brother Kimbal.: 6.857142857142858,
 The startup was acquired by Compaq for $307 million in 1999.: 0.857142857142857,
 The same year, Musk co-founded online bank X.com, which merged with Confinity in 2000 to form PayPal.: 4.2857142857142865,
 The company was bought by eBay in 2002 for $1.5 billion.: 2.714285714285714,
 
 
 In 2002, Musk founded SpaceX, an aerospace manufacturer and space transport services company, o

In [210]:
from heapq import nlargest

In [211]:
select_length = int(len(sentence_scores)*0.3)
select_length

5

In [212]:
summary = nlargest(select_length,sentence_scores,key = sentence_scores.get )

In [213]:
summary

[In 2016, he co-founded Neuralink, a neurotechnology company focused on developing brain–computer interfaces, and founded The Boring Company, a tunnel construction company.,
 
 
 In 2002, Musk founded SpaceX, an aerospace manufacturer and space transport services company, of which he is CEO and CTO.,
 He was enrolled at Queen's University and transferred to the University of Pennsylvania two years later, where he received a bachelor's degree in economics and physics, then moved to California in 1995 to attend Stanford University but decided instead to pursue a business career, co-founding the web software company Zip2 with his brother Kimbal.,
 In 2015, he co-founded OpenAI, a nonprofit research company that promotes friendly artificial intelligence.,
 In 2006, he helped create SolarCity, a solar energy services company that was later acquired by Tesla and became Tesla Energy.]

In [214]:
final_summary = [word.text for word in summary ]

In [215]:
summary = ''.join(final_summary)

In [216]:
print(summary)

In 2016, he co-founded Neuralink, a neurotechnology company focused on developing brain–computer interfaces, and founded The Boring Company, a tunnel construction company.

In 2002, Musk founded SpaceX, an aerospace manufacturer and space transport services company, of which he is CEO and CTO.He was enrolled at Queen's University and transferred to the University of Pennsylvania two years later, where he received a bachelor's degree in economics and physics, then moved to California in 1995 to attend Stanford University but decided instead to pursue a business career, co-founding the web software company Zip2 with his brother Kimbal.In 2015, he co-founded OpenAI, a nonprofit research company that promotes friendly artificial intelligence.In 2006, he helped create SolarCity, a solar energy services company that was later acquired by Tesla and became Tesla Energy.


In [217]:

context = summary
text = "paraphrase: "+context + " </s>"
encoding = tokenizer.encode_plus(text,max_length =128, padding=True, return_tensors="pt")
input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
model.eval()
diverse_beam_outputs = model.generate(
    input_ids=input_ids,attention_mask=attention_mask,
    max_length=128,
    early_stopping=True,
    num_beams=5,
    num_beam_groups = 5,
    num_return_sequences=5,
    diversity_penalty = 0.70
)
for beam_output in diverse_beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
result = sent[19:]


/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:197: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect.`max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`,or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect."
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2451: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2474: UserWarning: __floordiv__ is deprecated, and its behavior will chang

In [218]:
print(result)

Musk co-founded Neuralink, a brain-computer interface firm that created brain-computer interfaces and established The Boring Company, a tunnel construction firm that later became Tesla. In 2005, he co-founded OpenAI, a nonprofit research group that promotes friendly artificial intelligence. He was enrolled at Queen's University and later became Tesla Energy...


In [219]:
with open('/content/Result.txt', 'w') as writefile:
    writefile.write(result)
print(result)

Musk co-founded Neuralink, a brain-computer interface firm that created brain-computer interfaces and established The Boring Company, a tunnel construction firm that later became Tesla. In 2005, he co-founded OpenAI, a nonprofit research group that promotes friendly artificial intelligence. He was enrolled at Queen's University and later became Tesla Energy...


In [220]:
from gtts import gTTS


In [224]:
  from IPython.display import Audio
  tts = gTTS(result)
  tts.save('1.wav')
  sound = '1.wav'
  Audio(sound, autoplay = True)